<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [2]:
%pip install -Uqqq rich openai tiktoken wandb tenacity

In [3]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)
import wandb
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [4]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Paste your OpenAI key from: https://platform.openai.com/account/api-keys
··········
OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [5]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "generation"})

wandb: Currently logged in as: johannshetti. Use `wandb login --relogin` to force relogin


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [6]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [7]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [8]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))

generate_and_print(system_prompt, user_prompt)

How can I troubleshoot and resolve issues with my W&B account login?

How do I track the performance of different machine learning models in Weights & Biases?

How can I view the historical training metrics and performance of my machine learning experiments in W&B?

"Why am I getting an error message when trying to track an experiment using W&B?"

"How do I save and export my visualizations in W&B?"

# Few Shot

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [9]:
# Test if examples.txt is present, download if not
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

--2023-08-30 19:44:51--  https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40595 (40K) [text/plain]
Saving to: ‘examples.txt’

examples.txt        100%[===================>]  39.64K  --.-KB/s    in 0.001s  

2023-08-30 19:44:51 (60.8 MB/s) - ‘examples.txt’ saved [40595/40595]



In [10]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "I have an config file apart from sweep_config file so when I try to run sweep agent, I am getting     
UnboundLocalError("local variable 'config' referenced before assignment")."

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [11]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: can I use the 
column subsample parameter config file for xgboost parameters through sweeps? does w&b provide different templates 
for model evaluation? ok honey, another question for you: I am trying to compare the performance of adding         
different features fro training in a regression problem. I am able to track the performance with each feature, but 
I am unable to see which result corresponds to which feature in the chart section. Currently I am doing:           
wandb.init(project="dialect-creep", config={})                                                                     

for extra_feature in unique_features:                                                                              

                                                                                                                   
 wandb.config.update({'extra_feature': extra_feature})                                                             
                                                                                                                   

But it is no working. What can I do? Let's think step by step Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [12]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

Sure! Let's think step by step.                                                                                    

To track the performance of adding different features for training in a regression problem, you can update the     
config dictionary for each feature with a unique key-value pair. However, it seems that the current approach is not
working for you. To troubleshoot the issue, let's go through the steps together.                                   

Firstly, when you execute wandb.init(), make sure to pass the reinit=True argument to create a new run for each    
iteration. This will ensure that each run is properly logged and displayed separately in the W&B interface.        

Next, inside the loop where you are updating the extra_feature in wandb.config, you can manually create a unique   
suffix for each feature by appending it to the run's name or ID. For example:                                      

                                                                                                                   
 run_name = wandb.run.name if wandb.run.name else wandb.run.id                                                     
                                                                                                                   
 for extra_feature in unique_features:                                                                             
     new_run_name = f"{run_name}_{extra_feature}"                                                                  
     wandb.run.name = new_run_name                                                                                 
     wandb.config.extra_feature = extra_feature                                                                    
                                                                                                                   
     # Rest of your code for training and tracking the performance                                                 
                                                                                                                   

By updating the wandb.run.name attribute with the current feature value, you can create unique run names for each  
feature. Additionally, you are updating the extra_feature key in the wandb.config dictionary with the current      
feature value.                                                                                                     

This way, you will have separate runs for each feature, and in the W&B interface, you will be able to analyze the  
performance corresponding to each feature by filtering and comparing the different runs.                           

Give it a try, and let me know if you have any further questions or issues!

What steps can I take to track the performance of different features in a regression problem using W&B, and ensure 
that the corresponding result is shown in the chart section?

Sure! Based on the user query, a possible support question could be:                                               

"I am trying to track the performance of different features for training in a regression problem using W&B.        
Although I am updating the configuration for each unique feature, I am unable to see which result corresponds to   
which feature in the chart section. What step am I missing or what can I do to address this issue?"

How can I track the performance of different features in a regression problem and correspond the results to each   
feature in the chart section using Weights & Biases?

How can I track the performance of different features in a regression problem using W&B but also display which     
result corresponds to which feature in the chart section? The current method I'm using with                        
wandb.init(project="dialect-creep", config={}) and wandb.config.update({'extra_feature': extra_feature}) is not    
working. Can you help me troubleshoot this?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [13]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

Cloning into 'edu'...
remote: Enumerating objects: 2493, done.
remote: Counting objects: 100% (1031/1031), done.
remote: Compressing objects: 100% (381/381), done.
remote: Total 2493 (delta 723), reused 858 (delta 642), pack-reused 1462
Receiving objects: 100% (2493/2493), 22.60 MiB | 6.83 MiB/s, done.
Resolving deltas: 100% (1419/1419), done.


In [14]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [15]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[1644, 803, 4179, 2093, 2940, 365, 2596, 1206, 537, 2529, 956]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [16]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [17]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [18]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documentation: a panel and how you visualize that data.       

Overview                                                                                                           

 1 Log data to W&B                                                                                                 
 2 Create a query                                                                                                  
 3 Customize the chart                                                                                             


                                                1. Log data to W&B                                                 

First, log data in your script. Use ]8;id=966635;../../../../guides/track/config.md\wandb.config]8;;\ for single points set at the beginning of training, like          
hyperparameters. Use ]8;id=624172;../../../../guides/track/log/intro.md\wandb.log()]8;;\ for multiple points over time, and log custom 2D arrays with wandb.Table(). We    
recommend logging up to 10,000 data points per logged key.                                                         

                                                                                                                   
 # Logging a custom table of data                                                                                  
 my_custom_data = [[x1, y1, z1], [x2, y2, z2]]                                                                     
 wandb.log({"custom_data_table": wandb.Table(data=my_custom_data,                                                  
                                 columns = ["x", "y", "z"])})                                                      
                                                                                                                   

]8;id=857690;https://bit.ly/custom-charts-colab\Try a quick example notebook]8;;\ to log the data tables, and in the next step we'll set up custom charts. See what the 
resulting charts look like in the ]8;id=80563;https://app.wandb.ai/demo-team/custom-charts/reports/Custom-Charts--VmlldzoyMTk5MDc\live report]8;;\.                                                                     


                                                 2. Create a query                                                 

Once you've logged data to visualize, go to your project page and click the + button to add a new panel, then      
select Custom Chart. You can follow along in ]8;id=120518;https://app.wandb.ai/demo-team/custom-charts\this workspace]8;;\.                                                       

🌆 ]8;id=776082;/images/app_ui/create_a_query.png\A new, blank custom chart ready to be configured]8;;\                                                                                                                    

                                                    Add a query                                                    

 1 Click summary and select historyTable to set up a new query pulling data from the run history.                  
 2 Type in the key where you logged the wandb.Table(). In the code snippet above, it was my_custom_table . In the  
   ]8;id=400199;https://bit.ly/custom-charts-colab\example notebook]8;;\, the keys are pr_curve and roc_curve.                                                          

                                                  Set Vega fields                                                  

Now that the query is loading in these columns, they're available as options to select in the Vega fields dropdown 
menus:                                                                                                             

🌆 ]8;id=389224;/images/app_ui/set_vega_fields.png\Pulling in columns from the query results to set Vega fiel

Let's generate 3 possible questions:

In [19]:
generate_and_print(system_prompt, generation_prompt, n=3)

Support Question: How do I log a custom table of data in W&B using Python?

Support question: How can I log a custom table of data using W&B?                                                  

Answer (from the provided fragment of W&B documentation): To log a custom table of data, you can use the           
wandb.log() function and provide a dictionary with the key as "custom_data_table" and the value as a wandb.Table   
object. The wandb.Table object should be initialized with your custom data and column names. Here's an example:    

                                                                                                                   
 my_custom_data = [[x1, y1, z1], [x2, y2, z2]]                                                                     
 wandb.log({"custom_data_table": wandb.Table(data=my_custom_data, columns=["x", "y", "z"])})                       
                                                                                                                   


What is the recommended number of data points to log per key using wandb.log()?

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [20]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4"

In [22]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [23]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [25]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [26]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [27]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [28]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
How can I log my results in my team profile and not in my personal profile? I am trying to integrate gradio block  
in wandb. Here are my last three lines of code                                                                     

run = wandb.init(project=prms.WANDB_PROJECT, entity=prms.WANDB_ENTITY, job_type="gradio") block.launch(share=True) 
block.integrate(wandb=wandb)                                                                                       

gradio app is launched and working, wandb run as well, but there is not logged block in wandb                      

shouldn't it be enough to call the iface.integrate(wandb=wandb)? I am using the Hugging Face trainer to train a    
GPT-2 model. How can I log in wandb an example of what the model generates in each evaluation? For instance, at    
step 300 I want the log an example of the text generated by the model, and then again at step 600, and so on.      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
a panel and how you visualize that data.                                                                           

Overview                                                                                                           

 1 Log data to W&B                                                                                                 
 2 Create a query                                                                                                  
 3 Customize the chart                                                                                             


                                                1. Log data to W&B                                                 

First, log data in your script. Use ]8;id=681330;../../../../guides/track/config.md\wandb.config]8;;\ for single points set at the beginning of training, like          
hyperparameters. Use ]8;id=905212;../../../../guides/track/log/intro.md\wandb.log()]8;;\ for multiple points over time, and log custom 2D arrays with wandb.Table(). We    
recommend logging up to 10,000 data points per logged key.                                                         

                                                                                                                   
 # Logging a custom table of data                                                                                  
 my_custom_data = [[x1, y1, z1], [x2, y2, z2]]                                                                     
 wandb.log({"custom_data_table": wandb.Table(data=my_custom_data,                                                  
                                 columns = ["x", "y", "z"])})                                                      
                                                                                                                   

]8;id=47758;https://bit.ly/custom-charts-colab\Try a quick example notebook]8;;\ to log the data tables, and in the next step we'll set up custom charts. See what the 
resulting charts look like in the ]8;id=60696;https://app.wandb.ai/demo-team/custom-charts/reports/Custom-Charts--VmlldzoyMTk5MDc\live report]8;;\.                                                                     


                                                 2. Create a query                                             

In [29]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [30]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None

    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [31]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

('\nThe user might have successfully logged data to a W&B project and now they would like to visualize this data in a custom way, specifically by creating a custom chart. They eventually want to share this custom chart with colleagues or keep it for future usage in this project or other projects. \n',
 '\nHow can I create a custom chart in Weights and Biases, and is it possible to save it for future use? \n',
 "\nSure, you can definitely create a custom chart and once you are content with the visualization, you can save it for future purposes. Here is a step-by-step guide on how to do it:\n\n1. Go to your project page and click the **`+`** button to add a new panel, then select **Custom Chart**.\n\n2. Set up a new query by clicking `summary` and selecting `historyTable` to pull data from the run history. Then in the key section, log the **wandb.Table()**. For example, if your key was `my_custom_table` , type that in.\n\n3. Once the query loads in columns, they'll be available as option

In [32]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

<Artifact generated_examples>

In [33]:
wandb.finish()

usage/completion_tokens,▁▃▁▇▆█▇▅▅▆██▆▇█
usage/elapsed_time,▁▃▂▆▅▇▆▅▅▆█▇▆▇█
usage/prompt_tokens,▁▂▅███▇█▇▆█████
usage/total_tokens,▁▃▃▇▇█▇▆▆▇██▇▇█
usage/completion_tokens,1777
usage/elapsed_time,42.03143
usage/prompt_tokens,924
usage/total_tokens,2701
